# 1. Data Cleaning

In [1]:
import pandas as pd

excel_2016 = ["bikeshare-ridership-2016-q3.xlsx", "bikeshare-ridership-2016-q4.xlsx"]
q_df_2016 = [pd.read_excel(file) for file in excel_2016]

csv_2017 = ["Bikeshare Ridership (2017 Q1).csv", "Bikeshare Ridership (2017 Q2).csv", 
              "Bikeshare Ridership (2017 Q3).csv", "Bikeshare Ridership (2017 Q4).csv"]
q_df_2017 = [pd.read_csv(file) for file in csv_2017]

In [2]:
for i in range(2):
     q_df_2017[i].drop(['from_station_id', 'to_station_id'], axis=1, inplace=True)
        
q_df = q_df_2016 + q_df_2017

# Get the year, month, and day for every trip
for i in range(6):
    q_df[i].drop(['trip_id'], axis=1, inplace=True)
    if i < 2:
        q_df[i]['year'] = 2016
    else:
        q_df[i]['year'] = 2017
    q_df[i]['month'] = [int(date.split('/')[1]) if (isinstance(date, str) and (i == 1 or i == 2 
                        or i == 3)) else int(date.split('/')[0]) if (isinstance(date, str) and 
                        ( i == 4 or i == 5)) else int(date.month) for date in q_df[i]
                        ['trip_start_time']] 
    q_df[i]['day'] = [int(date.split('/')[0]) if (isinstance(date, str) and (i == 1 or i == 2 
                        or i == 3)) else int(date.split('/')[1]) if (isinstance(date, str) 
                        and (i == 4 or i == 5)) else int(date.day) for date in q_df[i]
                        ['trip_start_time']] 

In [3]:
# Scrape the weather data

years = [2016 for i in range (6)] + [2017 for i in range(12)]
months = [i for i in range(7, 13)] + [i for i in range(1, 13)]
date_tups = list(zip(months, years))

weather_urls = ["https://climate.weather.gc.ca/climate_data/daily_data_e.html?\
hlyRange=2002-06-04%7C2020-02-04&dlyRange=2002-06-04%7C2020-02-03&mlyRange=2003-07\
-01%7C2006-12-01&StationID=31688&Prov=ON&urlExtension=_e.html&searchType=stnProx&\
optLimit=yearRange&Month=" + str(month) + "&Day=1&StartYear=1840&EndYear=2020&Year=" + 
str(year) + "&selRowPerPage=25&Line=5&txtRadius=25&optProxType=city&selCity=43%7C39%7C7\
9%7C23%7CToronto&selPark=&txtCentralLatDeg=&txtCentralLatMin=0&txtCentralLatSec=0\
&txtCentralLongDeg=&txtCentralLongMin=0&txtCentralLongSec=0&txtLatDecDeg=&txtLong\
DecDeg=&timeframe=2" for (month, year) in date_tups]

weather_dfs = []

for link in weather_urls:
    weather_data = pd.read_html(link)
    weather_dfs.append(weather_data[0])

In [4]:
# Remove duration outliers
"""rides_df2 = rides_df1[rides_df1.from_station_name != rides_df1.to_station_name]

trip_durations = rides_df2['trip_duration_seconds']
iqr = trip_durations.quantile(0.75) - trip_durations.quantile(0.25)
    
upper_bound = trip_durations.quantile(0.75) + 1.5 * iqr
    
# Remove all outliers from the upper extreme.
rides_df3 = rides_df2[rides_df2.trip_duration_seconds <= upper_bound]
# Remove all outliers from the lower extreme.
rides_df4 = rides_df3[rides_df3.trip_duration_seconds >= 47]
rides_df5 = rides_df4.reset_index().drop(['index'], axis = 1)"""

"rides_df2 = rides_df1[rides_df1.from_station_name != rides_df1.to_station_name]\n\ntrip_durations = rides_df2['trip_duration_seconds']\niqr = trip_durations.quantile(0.75) - trip_durations.quantile(0.25)\n    \nupper_bound = trip_durations.quantile(0.75) + 1.5 * iqr\n    \n# Remove all outliers from the upper extreme.\nrides_df3 = rides_df2[rides_df2.trip_duration_seconds <= upper_bound]\n# Remove all outliers from the lower extreme.\nrides_df4 = rides_df3[rides_df3.trip_duration_seconds >= 47]\nrides_df5 = rides_df4.reset_index().drop(['index'], axis = 1)"

In [5]:
trips = pd.concat(q_df)
trips

,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_name,to_station_name,user_type,year,month,day
0,2016-07-09 01:03:00,2016-07-09 01:15:00,714,Dundas St E / Regent Park Blvd,Danforth Ave / Ellerbeck St,Member,2016,7,9
1,2016-07-09 02:15:00,2016-07-09 02:22:00,417,Riverdale Park North (Broadview Ave),Dundas St E / Regent Park Blvd,Member,2016,7,9
2,2016-07-10 17:04:00,2016-07-10 17:36:00,1904,Dundas St E / Regent Park Blvd,Queen St W / Close Ave,Member,2016,7,10
3,2016-07-11 01:45:00,2016-07-11 01:58:00,784,Union Station,Dundas St E / Regent Park Blvd,Member,2016,7,11
4,2016-07-18 13:35:00,2016-07-18 13:42:00,443,Front St W / Blue Jays Way,Front St / Yonge St (Hockey Hall of Fame),Member,2016,7,18
5,2016-07-18 15:48:00,2016-07-18 16:01:00,765,Union Station,Queens Park / Bloor St W,Member,2016,7,18
6,2016-07-18 16:29:00,2016-07-18 16:45:00,949,Queens Park / Bloor St W,Union Station,Member,2016,7,18
7,2016-07-19 21:15:00,2016-07-19 21:18:00,213,Simcoe St / Wellington St W,Widmer St / Adelaide St,Member,2016,7,19
8,2016-07-20 13:12:00,2016-07-20 13:23:00,658,Dundas St E / Regent Park Blvd,Union Station,Member,2016,7,20
9,2016-07-21 03:20:00,2016-07-21 03:40:00,1163,Danforth Ave / Lamb Ave,Dundas St E / Parliament St,Member,2016,7,21


# 2. Trip Location Visualizations

In [6]:
# Get the longitude and latitude for each station

import urllib
import json

url = "https://tor.publicbikesystem.net/ube/gbfs/v1/en/station_information"
response = urllib.request.urlopen(url)
station_dict = json.loads(response.read())
station_df = pd.DataFrame(station_dict['data']['stations'])

In [7]:

trips['longitude'] = [station_coords[start_station][0] if start_station in station_coords.keys()
                      else 'N/A' for start_station in trips['from_station_name']]
trips['latitude'] = [station_coords[start_station][1] if start_station in station_coords.keys()
                      else 'N/A' for start_station in trips['from_station_name']]

trips

,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_name,to_station_name,user_type,year,month,day,longitude,latitude
0,2016-07-09 01:03:00,2016-07-09 01:15:00,714,Dundas St E / Regent Park Blvd,Danforth Ave / Ellerbeck St,Member,2016,7,9,-79.3613,43.6602
1,2016-07-09 02:15:00,2016-07-09 02:22:00,417,Riverdale Park North (Broadview Ave),Dundas St E / Regent Park Blvd,Member,2016,7,9,-79.3547,43.6712
2,2016-07-10 17:04:00,2016-07-10 17:36:00,1904,Dundas St E / Regent Park Blvd,Queen St W / Close Ave,Member,2016,7,10,-79.3613,43.6602
3,2016-07-11 01:45:00,2016-07-11 01:58:00,784,Union Station,Dundas St E / Regent Park Blvd,Member,2016,7,11,-79.3804,43.6456
4,2016-07-18 13:35:00,2016-07-18 13:42:00,443,Front St W / Blue Jays Way,Front St / Yonge St (Hockey Hall of Fame),Member,2016,7,18,-79.3905,43.6435
5,2016-07-18 15:48:00,2016-07-18 16:01:00,765,Union Station,Queens Park / Bloor St W,Member,2016,7,18,-79.3804,43.6456
6,2016-07-18 16:29:00,2016-07-18 16:45:00,949,Queens Park / Bloor St W,Union Station,Member,2016,7,18,N/A,N/A
7,2016-07-19 21:15:00,2016-07-19 21:18:00,213,Simcoe St / Wellington St W,Widmer St / Adelaide St,Member,2016,7,19,N/A,N/A
8,2016-07-20 13:12:00,2016-07-20 13:23:00,658,Dundas St E / Regent Park Blvd,Union Station,Member,2016,7,20,-79.3613,43.6602
9,2016-07-21 03:20:00,2016-07-21 03:40:00,1163,Danforth Ave / Lamb Ave,Dundas St E / Parliament St,Member,2016,7,21,-79.3295,43.682


In [71]:
from collections import defaultdict

stations1 = [start_station if start_station in station_coords.keys()
                      else 0 for start_station in trips['from_station_name']]
stations2 = list(filter((0).__ne__, stations1))
stations_dict = defaultdict(int)
station_coords = {row['name']: (row['lon'], row['lat']) for _, row in station_df.iterrows()} 
for station in stations2:
    stations_dict[station] += 1
trips2 = pd.DataFrame({'Station': [station for station in stations_dict.keys()],
                       'Longitude': [station_coords[station][0] for 
                                     station in stations_dict.keys()],
                      'Latitude': [station_coords[station][1] for 
                                   station in stations_dict.keys()],
                      'Trips': [stations_dict[station] for 
                                station in stations_dict.keys()]})

In [93]:
import plotly.express as px

fig = px.density_mapbox(trips2, hover_name='Station', lon='Longitude', lat='Latitude', z='Trips', 
                        mapbox_style="open-street-map", 
                        zoom=11.5, radius=16)
fig.update_layout(title_text='Location of Bike Share Trips', title_x=0.5)
fig.show()